# Etapa 1

Gera tabela com genes e subtipos moleculares

In [1]:
import pandas as pd

C:\Users\IA\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\IA\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1.1 - Carrega tabelas de dados clínicos e genéticos

In [2]:
# Carregar o primeiro arquivo CSV com as colunas 'Sample ID' e 'Subtype'
clinical_data = pd.read_csv('Dados STAD clínico e patológico TCGA doutorado Julio.xlsx - data.csv', usecols=['Sample ID', 'Subtype'])

# Carregar o segundo arquivo CSV com as colunas 'Sample_ID', 'gene' e 'effect'
somatic_data = pd.read_csv('TCGA-STAD.varscan2_snv - TCGA-STAD.varscan2_snv.csv', usecols=['Sample_ID', 'gene', 'effect'])

### Informações sobre o conjunto de dados

**Subtipos Moleculares**

In [3]:
clinical_data.head()

,Sample ID,Subtype
0,TCGA-BR-8077-01,STAD_CIN
1,TCGA-BR-8289-01,STAD_CIN
2,TCGA-BR-8291-01,STAD_CIN
3,TCGA-F1-6875-01,STAD_CIN
4,TCGA-D7-8574-01,STAD_GS


In [4]:
clinical_data = clinical_data.dropna()
n_cases = len(clinical_data['Sample ID'].unique())
print(f"Quanditade de casos diferentes: {n_cases}")
print('Quantidade da casos para cada subtipo:')
print(clinical_data['Subtype'].value_counts())

Quanditade de casos diferentes: 383
Quantidade da casos para cada subtipo:
Subtype
STAD_CIN     223
STAD_MSI      73
STAD_GS       50
STAD_EBV      30
STAD_POLE      7
Name: count, dtype: int64


In [5]:
n_cin_cases = clinical_data['Subtype'].value_counts()['STAD_CIN']
n_msi_cases = clinical_data['Subtype'].value_counts()['STAD_MSI']
n_gs_cases = clinical_data['Subtype'].value_counts()['STAD_GS']
n_ebv_cases = clinical_data['Subtype'].value_counts()['STAD_EBV']
n_pole_cases = clinical_data['Subtype'].value_counts()['STAD_POLE']
print(f'''Portcentagem por classe: 
cin: {100*n_cin_cases/n_cases:.2f}% 
msi: {100*n_msi_cases/n_cases:.2f}% 
gs:  {100*n_gs_cases/n_cases:.2f}% 
ebv:  {100*n_ebv_cases/n_cases:.2f}% 
pole: {100*n_pole_cases/n_cases:.2f}%''')

Portcentagem por classe: 
cin: 58.22% 
msi: 19.06% 
gs:  13.05% 
ebv:  7.83% 
pole: 1.83%


**SNV**

In [6]:
somatic_data.head()

,gene,Sample_ID,effect
0,CLSTN1,TCGA-BR-7723-01A,missense_variant
1,PNRC2,TCGA-BR-7723-01A,frameshift_variant
2,AGBL4,TCGA-BR-7723-01A,intron_variant
3,LRRIQ3,TCGA-BR-7723-01A,missense_variant
4,TNNI3K,TCGA-BR-7723-01A,missense_variant


In [7]:
print(f"Quanditade total de amostras: {len(somatic_data['Sample_ID'])}")
print(f"Quantidade de casos diferentes: {len(somatic_data['Sample_ID'].unique())}")

Quanditade total de amostras: 187748
Quantidade de casos diferentes: 433


## 1.2 - Altera nome da coluna `Semple_ID` para mesclar as duas tabelas

In [8]:
# Removendo os ultimo caracteres do formato TCGA-SW-A7EB-01
clinical_data['Sample ID'] = clinical_data['Sample ID'].str[:12]

clinical_data.head()

,Sample ID,Subtype
0,TCGA-BR-8077,STAD_CIN
1,TCGA-BR-8289,STAD_CIN
2,TCGA-BR-8291,STAD_CIN
3,TCGA-F1-6875,STAD_CIN
4,TCGA-D7-8574,STAD_GS


In [9]:
# Renomear a coluna 'Sample_ID' no segundo DataFrame para 'Sample ID' para facilitar o merge
somatic_data = somatic_data.rename(columns={'Sample_ID': 'Sample ID'})

#Removendo o ultimo caractere do formato TCGA-SW-A7EB-01A
somatic_data['Sample ID'] = somatic_data['Sample ID'].str[:12]
somatic_data.head()

,gene,Sample ID,effect
0,CLSTN1,TCGA-BR-7723,missense_variant
1,PNRC2,TCGA-BR-7723,frameshift_variant
2,AGBL4,TCGA-BR-7723,intron_variant
3,LRRIQ3,TCGA-BR-7723,missense_variant
4,TNNI3K,TCGA-BR-7723,missense_variant


## 1.3 - Mescla as duas tabelas 

In [15]:
# Mesclar os DataFrames com base na coluna 'Sample ID'
merged_data = pd.merge(clinical_data, somatic_data, on='Sample ID', how='inner')

merged_data.head()

,Sample ID,Subtype,gene,effect
0,TCGA-BR-8077,STAD_CIN,MACF1,missense_variant
1,TCGA-BR-8077,STAD_CIN,GTF2B,missense_variant
2,TCGA-BR-8077,STAD_CIN,NBPF9,missense_variant
3,TCGA-BR-8077,STAD_CIN,S100A7L2,missense_variant
4,TCGA-BR-8077,STAD_CIN,APOA2,synonymous_variant


### 1.4 - Fazendo operações na tabela mesclada

As amostras que possuem na coluna `effect` o valor `synonymous_variant` são irrelevantes para esse estudo. Por isso serão exluida todas as amostras com esse valor. 

In [11]:
# Excluir os que são effect = 'synonymous_variant'
snp_subtype_missense_data = merged_data[merged_data['effect'] != 'synonymous_variant']

print(f"Foram excluidas: {len(somatic_data['Sample ID']) - len(snp_subtype_missense_data['Sample ID'])} amostras \nRestando:")
print(f"- Casos: {len(snp_subtype_missense_data['Sample ID'].unique())}")
print(f"- Amostras: {len(snp_subtype_missense_data['Sample ID'])}")

Foram excluidas: 57946 amostras 
Restando:
- Casos: 378
- Amostras: 129802


## 1.5 - Transoforma genes em colunas 

Transforma a tabela para que os genes se tornem coluna e que a tabela expresse a presença de cada gene por subtipo molecular.

In [16]:
df = snp_subtype_missense_data.drop(columns=['effect'])

# Criar uma tabela pivotada onde os genes se tornam colunas com 1 ou 0
df_pivot = df.pivot_table(index=['Sample ID', 'Subtype'], columns='gene', aggfunc=lambda x: 1, fill_value=0)

# Redefinir o índice para transformar em um DataFrame simples
df_final = df_pivot.reset_index()

In [17]:
# Removendo os casos POLE que não são utilizados no estudo
n_pole = df_final['Subtype'] != 'STAD_POLE'
df_final = df_final[n_pole]
print(df_final['Subtype'].value_counts())
print(f'Dimensções finais do dataset: {df_final.shape}')

# Renomeando subtipos para minúsculo e retirando 'STAD_'
df_final['Subtype'] = df_final['Subtype'].apply((lambda x: x.split('_')[1].lower()))
df_final.head()

Subtype
STAD_CIN    223
STAD_MSI     73
STAD_GS      45
STAD_EBV     30
Name: count, dtype: int64
Dimensções finais do dataset: (371, 18293)


gene,Sample ID,Subtype,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,ago.1,ago.2,ago.3,ago.4,pk
0,TCGA-3M-AB46,cin,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TCGA-3M-AB47,gs,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TCGA-B7-5816,msi,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,TCGA-B7-5818,ebv,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TCGA-B7-A5TI,msi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_final.to_csv('outputs/gene_subtype_table.csv')